In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import json
import dotenv
import requests
import os
dotenv.load_dotenv()



True

In [2]:
ct = contrans()
congresskey = os.getenv('congresskey')

In [15]:
members = ct.get_bioguideIDs()
replace_map = {'Republican': 'R', 'Democratic': 'D', 'Independent': 'I'}
members['partyletter'] = members['partyName'].replace(replace_map)
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,partyletter
0,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star...",R
1,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",D
2,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star...",D
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]",D
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star...",D
...,...,...,...,...,...,...,...,...,...,...,...
534,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",D
535,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",R
536,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",R
537,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",D


In [13]:
members['lastname'] = [x.split(',')[0] for x in members['name']]
members['firstname'] = [x.split(',')[1] for x in members['name']]
members['name2'] = [x + ' ' + y  + ' (' + z + ')' for x, y, z in zip(members['firstname'], members['lastname'],members['partyletter'] )]
members['name2']

0              Greg Lopez (R)
1            Kevin Mullin (D)
2         LaMonica McIver (D)
3         George S. Helmy (D)
4               Raul Ruiz (D)
                ...          
534     Michael F. Bennet (D)
535      Marsha Blackburn (R)
536          John Boozman (R)
537         Tammy Baldwin (D)
538         Chris Stewart (R)
Name: name2, Length: 539, dtype: object

In [6]:
cands = pd.read_csv('data/cands22.txt', quotechar="|", header=None)
cands.columns = ['Cycle', 'FECCandID', 'CID','FirstLastP','Party','DistIDRunFor','DistIDCurr','CurrCand','CycleCand','CRPICO','RecipCode','NoPacs']
cands

,Cycle,FECCandID,CID,FirstLastP,Party,DistIDRunFor,DistIDCurr,CurrCand,CycleCand,CRPICO,RecipCode,NoPacs
0,2022,H0AK00105,N00039029,Thomas Lamb (3),3,AK01,,,,,3N,
1,2022,H0AL01055,N00044245,Jerry Carl (R),R,AL01,AL01,Y,Y,I,RW,
2,2022,H0AL01063,N00044288,Wes Lambert (R),R,AL01,,,,,RN,
3,2022,H0AL01097,N00044750,James Averhart (D),D,AL01,,,,,DN,
4,2022,H0AL02087,N00030768,Martha Roby (R),R,AL02,,,,,RN,
...,...,...,...,...,...,...,...,...,...,...,...,...
8923,2022,S8WI00224,N00041929,Leah Vukmir (R),R,WIS1,,,,,RN,
8924,2022,S8WV00119,N00041293,Paula Jean Swearengin (D),D,WVS2,,,,,DN,
8925,2022,S8WV00127,N00035531,Evan Jenkins (R),R,WVS1,,,,,RN,
8926,2022,S8WV00135,N00041474,Bo Copley (R),R,WVS1,,,,,RN,


In [16]:
crosswalk = pd.merge(members, cands, left_on='name2', right_on='FirstLastP', how='outer', indicator = 'matched', validate='one_to_one')

KeyError: 'name2'

In [ ]:
# What kind of merge is this? One to one?

# What rows aren't matched and how can we fix it?

In [3]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."


In [4]:
goodid = goodbioguide.reset_index()['bioguideId'][0]

In [5]:
goodsbills = ct.get_sponseredLegislation(goodid)


In [6]:
billurl = goodsbills[0]['url'] #this is an example of one bill

In [ ]:
ct.get_billdata(billurl)

In [8]:
r = requests.get(goodsbills[0]['url'],
                params= {'api_key': congresskey})
#json.loads(r.text)
texturl = json.loads(r.text)['bill']['textVersions']['url']

r = requests.get(texturl,
                params= {'api_key': congresskey})

toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [9]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser')
billtext = mysoup.text

In [ ]:
mysoup.find_all('pre')

In [ ]:
print(mysoup.text)